## SQL Window Functions in SQLite

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import subprocess
import google.colab # if colab exists, install pysqlite-binary
subprocess.run(['pip', 'install', 'pysqlite3-binary'], capture_output=False)
import pysqlite3 as sqlite3
print(f'SQLite version: {sqlite3.sqlite_version}')

In [ ]:
conn = sqlite3.connect("/content/drive/MyDrive/0479/data/salaries.db") 

In [ ]:
import pandas as pd

## San Francisco Employee Salary Data (from Kaggle)


In [ ]:
pd.read_sql_query("select * from Salaries",conn)

<ipython-input-5-8a4e492e2099>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query("select * from Salaries",conn)


,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.00,400184.25,None,567595.43,567595.43,2011,None,San Francisco,None
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,None,538909.28,538909.28,2011,None,San Francisco,None
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.60,None,335279.91,335279.91,2011,None,San Francisco,None
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.00,56120.71,198306.90,None,332343.61,332343.61,2011,None,San Francisco,None
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.60,9737.00,182234.59,None,326373.19,326373.19,2011,None,San Francisco,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148645,148646,Carolyn A Wilson,Human Services Technician,0.00,0.00,0.00,0.00,0.00,0.00,2014,None,San Francisco,PT
148646,148648,Joann Anderson,Communications Dispatcher 2,0.00,0.00,0.00,0.00,0.00,0.00,2014,None,San Francisco,PT
148647,148649,Leon Walker,Custodian,0.00,0.00,0.00,0.00,0.00,0.00,2014,None,San Francisco,PT
148648,148650,Roy I Tillery,Custodian,0.00,0.00,0.00,0.00,0.00,0.00,2014,None,San Francisco,PT


## Query to Rank Employees Within Job Title by Descending Salary

Note that while this query groups (partitions) the data, it does not reduce the number of rows like a "group by" (aggregation) would.

In [ ]:
q='''
select EmployeeName, JobTitle, TotalPay, rank() 
over (partition by JobTitle order by TotalPay desc) as pay_rank 
from Salaries
'''
pd.read_sql_query(q,conn)

<ipython-input-6-023d5cd39ed0>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(q,conn)


,EmployeeName,JobTitle,TotalPay,pay_rank
0,NAPOLEON VIRAY,ACCOUNT CLERK,60838.20,1
1,BERNADETTE RAMOS,ACCOUNT CLERK,60530.87,2
2,SUSAN CHANG,ACCOUNT CLERK,58527.20,3
3,OLIVIA BOUDREAUX,ACCOUNT CLERK,54674.59,4
4,LI-XIN CAI,ACCOUNT CLERK,51792.25,5
...,...,...,...,...
148645,Mario Yedidia,Youth Comm Advisor,57544.73,1
148646,Phimy Truong,Youth Comm Advisor,52433.88,2
148647,Phimy Truong,Youth Comm Advisor,36465.91,3
148648,Mario R Yedidia,Youth Comm Advisor,19212.71,4


## Find Quintiles on Total Pay within Job Title

In [ ]:
q='''
select EmployeeName, JobTitle, TotalPay, ntile(5) over (partition by JobTitle order by TotalPay desc) 
quintile from Salaries order by JobTitle, quintile asc
'''
df=pd.read_sql_query(q,conn)
df

<ipython-input-7-e41e96fbd471>:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(q,conn)


,EmployeeName,JobTitle,TotalPay,quintile
0,NAPOLEON VIRAY,ACCOUNT CLERK,60838.20,1
1,BERNADETTE RAMOS,ACCOUNT CLERK,60530.87,1
2,SUSAN CHANG,ACCOUNT CLERK,58527.20,1
3,OLIVIA BOUDREAUX,ACCOUNT CLERK,54674.59,1
4,LI-XIN CAI,ACCOUNT CLERK,51792.25,1
...,...,...,...,...
148645,Mario Yedidia,Youth Comm Advisor,57544.73,1
148646,Phimy Truong,Youth Comm Advisor,52433.88,2
148647,Phimy Truong,Youth Comm Advisor,36465.91,3
148648,Mario R Yedidia,Youth Comm Advisor,19212.71,4


## Find salary difference between successive employees sorted by salary within job title

In [ ]:
q='''
select sf2.EmployeeName, JobTitle, TotalPay, next_salary, TotalPay-next_salary as salary_difference from 
(select EmployeeName, JobTitle, TotalPay, lead(TotalPay,1) 
over (partition by JobTitle order by TotalPay desc) 
as next_salary from Salaries) as sf2
'''
pd.read_sql_query(q,conn)

<ipython-input-8-df8b8bda3a8b>:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(q,conn)


,EmployeeName,JobTitle,TotalPay,next_salary,salary_difference
0,NAPOLEON VIRAY,ACCOUNT CLERK,60838.20,60530.87,307.33
1,BERNADETTE RAMOS,ACCOUNT CLERK,60530.87,58527.20,2003.67
2,SUSAN CHANG,ACCOUNT CLERK,58527.20,54674.59,3852.61
3,OLIVIA BOUDREAUX,ACCOUNT CLERK,54674.59,51792.25,2882.34
4,LI-XIN CAI,ACCOUNT CLERK,51792.25,51670.27,121.98
...,...,...,...,...,...
148645,Mario Yedidia,Youth Comm Advisor,57544.73,52433.88,5110.85
148646,Phimy Truong,Youth Comm Advisor,52433.88,36465.91,15967.97
148647,Phimy Truong,Youth Comm Advisor,36465.91,19212.71,17253.20
148648,Mario R Yedidia,Youth Comm Advisor,19212.71,NaN,NaN


## Find pay rank among all employees irrespective of title (window is entire table, sorted)

In [ ]:
q='''
select EmployeeName, JobTitle, TotalPay, rank() 
over (order by TotalPay desc) as pay_rank from salaries
'''
pd.read_sql_query(q,conn)

<ipython-input-9-284b1a9e664f>:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(q,conn)


,EmployeeName,JobTitle,TotalPay,pay_rank
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,567595.43,1
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),538909.28,2
2,David Shinn,Deputy Chief 3,471952.64,3
3,Amy P Hart,Asst Med Examiner,390111.98,4
4,Gary Altenberg,"Lieutenant, Fire Suppression",362844.66,5
...,...,...,...,...
148645,Carolyn A Wilson,Human Services Technician,0.00,148286
148646,Joann Anderson,Communications Dispatcher 2,0.00,148286
148647,Leon Walker,Custodian,0.00,148286
148648,Roy I Tillery,Custodian,0.00,148286


## Find average total pay within the global quintiles

In [ ]:
q='''
select *, avg(TotalPay) over (partition by quintile) as quintileAvgPay
from (select EmployeeName, JobTitle, TotalPay, ntile(5) 
over (order by TotalPay desc) as quintile from salaries) 
'''
df=pd.read_sql_query(q,conn)
df

<ipython-input-10-ee6fca0b4c55>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(q,conn)


,EmployeeName,JobTitle,TotalPay,quintile,quintileAvgPay
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,567595.43,1,150603.247625
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),538909.28,1,150603.247625
2,David Shinn,Deputy Chief 3,471952.64,1,150603.247625
3,Amy P Hart,Asst Med Examiner,390111.98,1,150603.247625
4,Gary Altenberg,"Lieutenant, Fire Suppression",362844.66,1,150603.247625
...,...,...,...,...,...
148645,Carolyn A Wilson,Human Services Technician,0.00,5,8390.054344
148646,Joann Anderson,Communications Dispatcher 2,0.00,5,8390.054344
148647,Leon Walker,Custodian,0.00,5,8390.054344
148648,Roy I Tillery,Custodian,0.00,5,8390.054344


## Find difference between total pay and average total pay within each quintile

In [ ]:
q='''
select *, TotalPay-AveragePay as pay_difference 
from (select *, avg(TotalPay) 
over (partition by quintile) as AveragePay 
from (select EmployeeName, JobTitle, TotalPay, ntile(5) 
over (order by TotalPay desc) as quintile from salaries) sf2) sf
'''
pd.read_sql_query(q,conn)

<ipython-input-11-08496541fd7a>:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(q,conn)


,EmployeeName,JobTitle,TotalPay,quintile,AveragePay,pay_difference
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,567595.43,1,150603.247625,416992.182375
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),538909.28,1,150603.247625,388306.032375
2,David Shinn,Deputy Chief 3,471952.64,1,150603.247625,321349.392375
3,Amy P Hart,Asst Med Examiner,390111.98,1,150603.247625,239508.732375
4,Gary Altenberg,"Lieutenant, Fire Suppression",362844.66,1,150603.247625,212241.412375
...,...,...,...,...,...,...
148645,Carolyn A Wilson,Human Services Technician,0.00,5,8390.054344,-8390.054344
148646,Joann Anderson,Communications Dispatcher 2,0.00,5,8390.054344,-8390.054344
148647,Leon Walker,Custodian,0.00,5,8390.054344,-8390.054344
148648,Roy I Tillery,Custodian,0.00,5,8390.054344,-8390.054344


## Average, Max, and Min Pay by Quintile

In [ ]:
# note: often the quintiles are presented in the opposite order
q='''
select distinct quintile, average_pay, max_pay, min_pay 
from (select *, avg(TotalPay) over (partition by quintile) as average_pay, 
max(TotalPay) over (partition by quintile) as max_pay, 
min(TotalPay) over (partition by quintile) as min_pay 
from (select EmployeeName, JobTitle, TotalPay, ntile(5) 
over (order by TotalPay desc) as quintile from salaries)) 
order by quintile
'''
pd.read_sql_query(q,conn)

<ipython-input-12-67c5ff0688ff>:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(q,conn)


,quintile,average_pay,max_pay,min_pay
0,1,150603.247625,567595.43,116209.16
1,2,97925.387716,116203.97,82595.59
2,3,71397.679481,82595.55,60708.58
3,4,45535.300339,60704.44,22563.00
4,5,8390.054344,22562.17,-618.13


In [ ]:
# window function to define the quintile
# aggregation to find the avg, max, min
q='''
select quintile, avg(TotalPay), max(TotalPay), min(TotalPay) from
(select EmployeeName, JobTitle, TotalPay, ntile(5) 
over (order by TotalPay desc) as quintile from salaries) 
group by quintile order by quintile
'''
pd.read_sql_query(q,conn)

<ipython-input-14-ab83df089dc0>:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(q,conn)


,quintile,avg(TotalPay),max(TotalPay),min(TotalPay)
0,1,150603.247625,567595.43,116209.16
1,2,97925.387716,116203.97,82595.59
2,3,71397.679481,82595.55,60708.58
3,4,45535.300339,60704.44,22563.00
4,5,8390.054344,22562.17,-618.13


## Olympic Athlete Data

In [ ]:
conn2 = sqlite3.connect("/content/drive/MyDrive/0479/data/athlete_events.db")

In [ ]:
pd.read_sql_query("select * from athlete_events",conn2)

<ipython-input-16-a851ab21a29c>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query("select * from athlete_events",conn2)


,id,name,sex,age,height,weight,team,noc,games,year,season,city,sport,event,medal
0,1,A Dijiang,M,24,180,80,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NA
1,2,A Lamusi,M,23,170,60,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NA
2,3,Gunnar Nielsen Aaby,M,24,NA,NA,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NA
3,4,Edgar Lindenau Aabye,M,34,NA,NA,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21,185,82,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29,179,89,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NA
271112,135570,Piotr ya,M,27,176,59,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NA
271113,135570,Piotr ya,M,27,176,59,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NA
271114,135571,Tomasz Ireneusz ya,M,30,185,96,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NA


## Make a view with summer medals only and selected fields

In [ ]:
q='''
create view summer_medals_only as 
select name, year, event, medal from athlete_events where season='Summer' 
and medal in ('Bronze','Gold','Silver') order by name, event, year
'''

In [ ]:
pd.read_sql_query("select * from summer_medals_only order by year desc",conn2)

<ipython-input-18-26afa34e8cae>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query("select * from summer_medals_only order by year desc",conn2)


,name,year,event,medal
0,Giovanni Abagnale,2016,Rowing Men's Coxless Pairs,Bronze
1,Patimat Abakarova,2016,Taekwondo Women's Flyweight,Bronze
2,Luc Abalo,2016,Handball Men's Handball,Silver
3,Saeid Morad Abdevali,2016,"Wrestling Men's Middleweight, Greco-Roman",Bronze
4,Denis Mikhaylovich Ablyazin,2016,Gymnastics Men's Team All-Around,Silver
...,...,...,...,...
34083,Hermann Otto Ludwig Weingrtner,1896,Gymnastics Men's Pommelled Horse,Silver
34084,Thomas Xenakis,1896,Gymnastics Men's Rope Climbing,Silver
34085,"Jules Alexis ""Louis"" Zutter",1896,Gymnastics Men's Horse Vault,Silver
34086,"Jules Alexis ""Louis"" Zutter",1896,Gymnastics Men's Parallel Bars,Silver


## Make a view with the medal before this one for that athlete, if any (not NULL)

In [ ]:
q='''
create view medal_lag as
select * from (select *, lag(medal,1)
over (partition by name, event order by name, event, year)
as lastmedal from summer_medals_only) l
where lastmedal is not NULL order by name, event, year
'''

In [ ]:
pd.read_sql_query("select * from  summer_medals_only where name='Ai Fukuhara'",conn2)

<ipython-input-20-0b564d1499ce>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query("select * from  summer_medals_only where name='Ai Fukuhara'",conn2)


,name,year,event,medal
0,Ai Fukuhara,2012,Table Tennis Women's Team,Silver
1,Ai Fukuhara,2016,Table Tennis Women's Team,Bronze


In [ ]:
pd.read_sql_query("select * from medal_lag order by year desc", conn2)

<ipython-input-21-02424641c90d>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query("select * from medal_lag order by year desc", conn2)


,name,year,event,medal,lastmedal
0,Adilbek Sabitovich Niyazymbetov,2016,Boxing Men's Light-Heavyweight,Silver,Silver
1,Ai Fukuhara,2016,Table Tennis Women's Team,Bronze,Silver
2,Aleksandra Viktorovna Gerasimenya,2016,Swimming Women's 50 metres Freestyle,Bronze,Silver
3,Aleksandra Vyacheslavovna Patskevich,2016,Synchronized Swimming Women's Team,Gold,Gold
4,"Alexander John ""Alex"" Gregory",2016,Rowing Men's Coxless Fours,Gold,Gold
...,...,...,...,...,...
5354,Robert Linzeler,1900,Sailing Mixed 0-0.5 Ton,Silver,Silver
5355,Texier I,1900,Sailing Mixed 0-0.5 Ton,Silver,Silver
5356,Texier II,1900,Sailing Mixed 0-0.5 Ton,Silver,Silver
5357,Victor Auguste Godinet,1900,Sailing Mixed 2-3 Ton,Silver,Silver


## Find only repeated gold

In [ ]:
q='''
select * from medal_lag where medal="Gold" and lastmedal="Gold" order by year desc
'''
pd.read_sql_query(q,conn2)

<ipython-input-24-ef00418adee4>:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(q,conn2)


,name,year,event,medal,lastmedal
0,Aleksandra Vyacheslavovna Patskevich,2016,Synchronized Swimming Women's Team,Gold,Gold
1,"Alexander John ""Alex"" Gregory",2016,Rowing Men's Coxless Fours,Gold,Gold
2,Alexander Mah Owens Drysdale,2016,Rowing Men's Single Sculls,Gold,Gold
3,"Alexandra Rose ""Aly"" Raisman",2016,Gymnastics Women's Team All-Around,Gold,Gold
4,Alistair Edward Brownlee,2016,Triathlon Men's Olympic Distance,Gold,Gold
...,...,...,...,...,...
1244,"Raymond Clarence ""Ray"" Ewry",1904,Athletics Men's Standing Long Jump,Gold,Gold
1245,"Raymond Clarence ""Ray"" Ewry",1904,Athletics Men's Standing Triple Jump,Gold,Gold
1246,Gaston Frdric Blanchy,1900,Sailing Mixed 2-3 Ton,Gold,Gold
1247,Jacques le Lavasseur,1900,Sailing Mixed 2-3 Ton,Gold,Gold


## Count number of consecutive gold medals per athlete

In [ ]:
q='''
select name,count(*)+1 as conseq_gold_count 
from medal_lag where medal='Gold' and 
lastmedal='Gold' group by name order by conseq_gold_count desc
'''
pd.read_sql_query(q,conn2)


<ipython-input-25-0d5686123708>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(q,conn2)


,name,conseq_gold_count
0,"Michael Fred Phelps, II",14
1,"Raymond Clarence ""Ray"" Ewry",8
2,Usain St. Leo Bolt,6
3,Larysa Semenivna Latynina (Diriy-),6
4,"Jennifer Elisabeth ""Jenny"" Thompson (-Cumpelik)",6
...,...,...
997,Adrien Gry Rommel,2
998,Adolf Dirk Coenraad van der Voort van Zijp,2
999,Adhemar Ferreira da Silva,2
1000,"Abraham Robertus ""Bram"" Lomans",2


## Microsoft Daily Stock Price Data, 1986-2020

In [ ]:
conn3 = sqlite3.connect("/content/drive/MyDrive/0479/data/msft.db")

In [ ]:
pd.read_sql_query("select * from msft",conn3)

<ipython-input-28-a45949590681>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query("select * from msft",conn3)


,date,open,high,low,close,adj_close,volume
0,1986-03-13,0.088542,0.101562,0.088542,0.097222,0.062378,1031788800
1,1986-03-14,0.097222,0.102431,0.097222,0.100694,0.064606,308160000
2,1986-03-17,0.100694,0.103299,0.100694,0.102431,0.065720,133171200
3,1986-03-18,0.102431,0.103299,0.098958,0.099826,0.064049,67766400
4,1986-03-19,0.099826,0.100694,0.097222,0.098090,0.062935,47894400
...,...,...,...,...,...,...,...
8579,2020-03-26,148.399994,156.660004,148.369995,156.110001,156.110001,64568100
8580,2020-03-27,151.750000,154.889999,149.199997,149.699997,149.699997,57042300
8581,2020-03-30,152.440002,160.600006,150.009995,160.229996,160.229996,63420300
8582,2020-03-31,159.399994,164.779999,156.559998,157.710007,157.710007,77927200


## Create an aggregated view containing monthly average volumes

In [ ]:
q='''
create view monthly_averages as 
select strftime('%Y',date) as year, 
strftime('%M',date) as month, avg(volume) as avg 
from msft 
group by year, month order by year, month
'''

In [ ]:
pd.read_sql_query("select * from monthly_averages", conn3)

<ipython-input-30-26a30032f067>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query("select * from monthly_averages", conn3)


,year,month,avg
0,1986,03,1.547544e+08
1,1986,04,2.487011e+07
2,1986,05,1.587840e+07
3,1986,06,1.398994e+07
4,1986,07,2.094545e+07
...,...,...,...
405,2019,12,2.144301e+07
406,2020,01,2.659667e+07
407,2020,02,4.671712e+07
408,2020,03,7.330434e+07


## Create a view with just the day, month and year and the volume

In [ ]:
q='''
create view volumes_only as
select strftime('%Y',date) as year,
strftime('%m',date) as month,
strftime('%d',date) as day, volume from msft
'''

In [ ]:
pd.read_sql_query("select * from volumes_only", conn3)

<ipython-input-32-55e07cfb3a33>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query("select * from volumes_only", conn3)


,year,month,day,volume
0,1986,03,13,1031788800
1,1986,03,14,308160000
2,1986,03,17,133171200
3,1986,03,18,67766400
4,1986,03,19,47894400
...,...,...,...,...
8579,2020,03,26,64568100
8580,2020,03,27,57042300
8581,2020,03,30,63420300
8582,2020,03,31,77927200


## Join the two views so as to see how much the daily volume varies from the monthly average

In [ ]:
q='''
select volumes_only.*, monthly_averages.avg as monthly_average_volume, 
round((volumes_only.volume)*100.0/monthly_averages.avg,2) as percentage_of_average 
from volumes_only inner join monthly_averages 
on volumes_only.year=monthly_averages.year and volumes_only.month=monthly_averages.month
'''
pd.read_sql_query(q,conn3)

<ipython-input-33-3a6db9ebef5e>:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(q,conn3)


,year,month,day,volume,monthly_average_volume,percentage_of_average
0,1986,03,13,1031788800,1.547544e+08,666.73
1,1986,03,14,308160000,1.547544e+08,199.13
2,1986,03,17,133171200,1.547544e+08,86.05
3,1986,03,18,67766400,1.547544e+08,43.79
4,1986,03,19,47894400,1.547544e+08,30.95
...,...,...,...,...,...,...
8579,2020,03,26,64568100,7.330434e+07,88.08
8580,2020,03,27,57042300,7.330434e+07,77.82
8581,2020,03,30,63420300,7.330434e+07,86.52
8582,2020,03,31,77927200,7.330434e+07,106.31


## Do the same thing, but with just one view and no join, using a window function and "partition by"

In [ ]:
q='''
select *, round(volume*100.0/monthly_average_volume,2) as percentage_of_average 
from 
(select *, avg(volume) over (partition by year, month) as monthly_average_volume 
from volumes_only) v
'''
pd.read_sql_query(q,conn3)

<ipython-input-34-0b20b8dfe133>:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(q,conn3)


,year,month,day,volume,monthly_average_volume,percentage_of_average
0,1986,03,13,1031788800,1.547544e+08,666.73
1,1986,03,14,308160000,1.547544e+08,199.13
2,1986,03,17,133171200,1.547544e+08,86.05
3,1986,03,18,67766400,1.547544e+08,43.79
4,1986,03,19,47894400,1.547544e+08,30.95
...,...,...,...,...,...,...
8579,2020,03,26,64568100,7.330434e+07,88.08
8580,2020,03,27,57042300,7.330434e+07,77.82
8581,2020,03,30,63420300,7.330434e+07,86.52
8582,2020,03,31,77927200,7.330434e+07,106.31


## Calculate running sums of volumes by month

In [ ]:
q='''
select *,sum(volume) over (partition by year, month order by year, month, day) as running_sum
from volumes_only
'''
pd.read_sql_query(q,conn3)

<ipython-input-35-39648509c44e>:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(q,conn3)


,year,month,day,volume,running_sum
0,1986,03,13,1031788800,1031788800
1,1986,03,14,308160000,1339948800
2,1986,03,17,133171200,1473120000
3,1986,03,18,67766400,1540886400
4,1986,03,19,47894400,1588780800
...,...,...,...,...,...
8579,2020,03,26,64568100,1414305700
8580,2020,03,27,57042300,1471348000
8581,2020,03,30,63420300,1534768300
8582,2020,03,31,77927200,1612695500


## Create a view of the closing prices only with year, month, and day

In [ ]:
q='''
create view closing_prices_only as
select strftime('%Y',date) as year,
strftime('%m',date) as month,
strftime('%d',date) as day, adj_close from msft
'''

## Add a column of the max closing price by month

In [ ]:
q='''
select *, max(adj_close) over (partition by year, month) as maxprice
from closing_prices_only order by year, month, day
'''
pd.read_sql_query(q,conn3)

<ipython-input-37-854cec366c32>:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(q,conn3)


,year,month,day,adj_close,maxprice
0,1986,03,13,0.062378,0.065720
1,1986,03,14,0.064606,0.065720
2,1986,03,17,0.065720,0.065720
3,1986,03,18,0.064049,0.065720
4,1986,03,19,0.062935,0.065720
...,...,...,...,...,...
8579,2020,03,26,156.110001,172.789993
8580,2020,03,27,149.699997,172.789993
8581,2020,03,30,160.229996,172.789993
8582,2020,03,31,157.710007,172.789993


## Rank by descending closing price within month

In [ ]:
q='''
select *,rank() over (partition by year, month order by adj_close desc) as r
from closing_prices_only where year='2020'
'''
pd.read_sql_query(q,conn3)

<ipython-input-38-23478c532eb6>:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(q,conn3)


,year,month,day,adj_close,r
0,2020,01,30,172.309357,1
1,2020,01,31,169.766296,2
2,2020,01,29,167.582260,3
3,2020,01,17,166.644836,4
4,2020,01,23,166.265869,5
...,...,...,...,...,...
58,2020,03,12,139.059998,19
59,2020,03,20,137.350006,20
60,2020,03,23,135.979996,21
61,2020,03,16,135.419998,22


## Find only the top price per month (there many be ties)

In [ ]:
q='''
select * from 
(select *,rank() 
over (partition by year, month 
order by adj_close desc) as r 
from closing_prices_only) prices_ranked where r=1
'''
pd.read_sql_query(q,conn3)

<ipython-input-39-3a3607d206ad>:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(q,conn3)


,year,month,day,adj_close,r
0,1986,03,17,0.065720,1
1,1986,04,28,0.075745,1
2,1986,05,30,0.077973,1
3,1986,06,05,0.076302,1
4,1986,06,06,0.076302,1
...,...,...,...,...,...
429,2019,12,27,158.527008,1
430,2020,01,30,172.309357,1
431,2020,02,10,188.185989,1
432,2020,03,02,172.789993,1


## Find a five-day rolling average of today's price and the four before

In [ ]:
q='''
select *, avg(adj_close) 
over (order by year,month,day rows between 4 preceding and current row) as rolling_average
from closing_prices_only
'''
pd.read_sql_query(q,conn3)

<ipython-input-40-3ef11ba3496f>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(q,conn3)


,year,month,day,adj_close,rolling_average
0,1986,03,13,0.062378,0.062378
1,1986,03,14,0.064606,0.063492
2,1986,03,17,0.065720,0.064235
3,1986,03,18,0.064049,0.064188
4,1986,03,19,0.062935,0.063938
...,...,...,...,...,...
8579,2020,03,26,156.110001,144.939999
8580,2020,03,27,149.699997,147.409998
8581,2020,03,30,160.229996,152.259998
8582,2020,03,31,157.710007,154.134000


## Create a view with a one day price lag

In [ ]:
q='''
create view one_day_price_lag as 
(select *, lag(adj_close,1) over(order by year, month, day)  as yclose 
from closing_prices_only)
'''

## Calculate the one-day return

In [ ]:
q='''
select *,(adj_close-yclose)*100/yclose as return from one_day_price_lag
'''
pd.read_sql_query(q,conn3)

<ipython-input-43-33cecf01825c>:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(q,conn3)


,year,month,day,adj_close,yclose,return
0,1986,03,13,0.062378,NaN,NaN
1,1986,03,14,0.064606,0.062378,3.571506
2,1986,03,17,0.065720,0.064606,1.724127
3,1986,03,18,0.064049,0.065720,-2.542403
4,1986,03,19,0.062935,0.064049,-1.739120
...,...,...,...,...,...,...
8579,2020,03,26,156.110001,146.919998,6.255107
8580,2020,03,27,149.699997,156.110001,-4.106081
8581,2020,03,30,160.229996,149.699997,7.034067
8582,2020,03,31,157.710007,160.229996,-1.572732
